In [1]:
"""Table Purpose: Calculate the length and width  of the orphan dendrites

Why useful: 
1) Save the restriction that gets the orphan dendrites
2) Finalize the width as the 95th percentile
3) Calculate the lengths of the skeletons






"""

'Table Purpose: Calculate the length and width  of the orphan dendrites\n\nWhy useful: \n1) Save the restriction that gets the orphan dendrites\n2) Finalize the width as the 95th percentile\n3) Calculate the lengths of the skeletons\n\n\n\n\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import time
import os
import datetime

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv
import pandas as pd
from tqdm import tqdm


#for filtering
import math
from pykdtree.kdtree import KDTree

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


Connecting celiib@10.28.0.34:3306


In [4]:
"""
Purpose of Table: 
-- setting the key source
1) Only gets the largest significant dendrite component for each orphan neuron
2) Uses that list to restrict DendriteWidth

-- what the make function does
1) fetches the 95th percent width of the component from the DendriteWidth table
2) Gets the skeleton of the orphan dendrite from the Filtered Skeleton page
3) Calculates the total length of the skeleton 
4) Pushes both the 95th percentil of min width and the skeleton length to table

"""

'\nPurpose of Table: \n-- setting the key source\n1) Only gets the largest significant dendrite component for each orphan neuron\n2) Uses that list to restrict DendriteWidth\n\n-- what the make function does\n1) fetches the 95th percent width of the component from the DendriteWidth table\n2) Gets the skeleton of the orphan dendrite from the Filtered Skeleton page\n3) Calculates the total length of the skeleton \n4) Pushes both the 95th percentil of min width and the skeleton length to table\n\n'

In [5]:
@schema
class SignificantOrphanDendriteWidthLength(dj.Computed):
    definition="""
    -> pinky.DendriteWidth
    ---
    width           :float #95th pecentile of min width range for final width
    skeleton_length :float #total length of skeleton
    """
    
    #calculating the significant orphan components for all of the ID's
    start_table = (pinky.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'").proj("n_vertex_indices")
    seg_ids = start_table.fetch("segment_id")
    len(seg_ids),len(set(seg_ids))

    component_total = []
    for s in set(seg_ids):
        seg_data = (start_table & "segment_id="+str(s)).fetch("component_index","n_vertex_indices")

        components = seg_data[0]
        n_vertices = seg_data[1]


        #get the maximum n_vertices dendrite
        n_verts_max = max(n_vertices)
        components_max = components[n_vertices == n_verts_max][0]


        new_dict = dict(segment_id=s,component_index=components_max,n_vertex_indices=n_verts_max)
        component_total.append(new_dict)
    
    key_source = (pinky.DendriteWidth() & "min_width_max>0" & pinky.CompartmentOrphan.ComponentOrphan.proj() & "compartment_type='Dendrite'" & component_total)

    def make(self, key):
        #with the key pull down the width
        width = (pinky.DendriteWidth & key).fetch1("min_width_perc_95")
        
        #*****pull down the skeleton for the mesh (should be from highly discretized list****
        skeleton_edges = (pinky.FilteredNeuronSkeleton() & key).fetch1("edges")
        skeleton_edges.shape

        column_1 = skeleton_edges[:,0,:]
        column_2 = skeleton_edges[:,1,:]

        skeleton_length = 0

        for a,b in zip(column_1,column_2):
            skeleton_length+= np.linalg.norm(a-b)
        
        #save to the database
        new_key = dict(key,width=width,skeleton_length=skeleton_length)
        
        self.insert1(new_key,skip_duplicates=True)
        

        
        
    
    

In [6]:
#(schema.jobs & "table_name='__significant_orphan_dendrite_width_length'").delete()

In [7]:
#SignificantOrphanDendriteWidthLength()

In [9]:
SignificantOrphanDendriteWidthLength()

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:150: UserWarning: Mysql server has gone away. Reconnecting to the server.
  warnings.warn("Mysql server has gone away. Reconnecting to the server.")


segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,discrete_length the maximum size of a discretized segment of postsyn skeleton,width 95th pecentile of min width range for final width,skeleton_length total length of skeleton
3,648518346341353058,0.35,Dendrite,0,10,336.509,130240.0
3,648518346341353607,0.35,Dendrite,0,10,628.359,146276.0
3,648518346341354313,0.35,Dendrite,0,10,634.06,95712.3
3,648518346341354496,0.35,Dendrite,0,10,913.721,100540.0
3,648518346341355048,0.35,Dendrite,0,10,350.602,323800.0
3,648518346341355198,0.35,Dendrite,0,10,690.183,29475.2
3,648518346341355736,0.35,Dendrite,0,10,310.327,211692.0
3,648518346341356241,0.35,Dendrite,0,10,345.933,70524.8
3,648518346341356256,0.35,Dendrite,0,10,466.534,1259830.0
3,648518346341357855,0.35,Dendrite,0,10,464.59,238230.0


In [10]:
pinky.DendriteWidth()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,discrete_length the maximum size of a discretized segment of postsyn skeleton,min_width_perc_50 50th pecentile min width range for all skeleton,min_width_perc_55 55th pecentile min width range for all skeleton,min_width_perc_60 60th pecentile min width range for all skeleton,min_width_perc_65 65th pecentile min width range for all skeleton,min_width_perc_70 70th pecentile min width range for all skeleton,min_width_perc_75 75th pecentile min width range for all skeleton,min_width_perc_80 80th pecentile min width range for all skeleton,min_width_perc_85 85th pecentile min width range for all skeleton,min_width_perc_90 90th pecentile min width range for all skeleton,min_width_perc_95 95th pecentile min width range for all skeleton,min_width_max max min width range for all skeleton
3,648518346341353058,0.35,Dendrite,0,10,187.424,202.255,213.753,226.635,241.619,256.967,274.097,291.2,311.229,336.509,418.514
3,648518346341353574,0.35,Basal,0,10,147.006,158.274,170.409,182.787,194.771,207.006,221.228,245.707,284.295,367.462,977.171
3,648518346341353607,0.35,Dendrite,0,10,375.449,398.624,421.694,444.547,469.475,497.439,528.679,560.104,594.016,628.359,781.866
3,648518346341354313,0.35,Dendrite,0,10,454.067,467.686,481.699,499.16,515.606,535.016,552.751,573.972,595.75,634.06,691.867
3,648518346341354496,0.35,Dendrite,0,10,697.216,716.681,741.023,759.52,775.863,792.268,811.303,835.734,872.918,913.721,1019.83
3,648518346341355048,0.35,Dendrite,0,10,196.664,208.02,221.525,235.18,248.631,262.076,277.278,296.597,318.276,350.602,535.247
3,648518346341355198,0.35,Dendrite,0,10,594.304,605.154,613.765,624.085,631.077,637.519,645.764,658.325,669.407,690.183,736.082
3,648518346341355736,0.35,Dendrite,0,10,191.165,202.886,213.01,222.834,232.759,242.523,254.188,266.73,286.279,310.327,438.808
3,648518346341356241,0.35,Dendrite,0,10,230.537,237.029,244.306,252.687,261.764,271.395,283.269,298.057,317.593,345.933,409.251
3,648518346341356256,0.35,Dendrite,0,10,156.304,169.707,183.894,198.478,213.457,231.678,255.412,290.501,359.642,466.534,816.083


In [8]:
start_time = time.time()
SignificantOrphanDendriteWidthLength.populate(reserve_jobs=True)
print(f"Total time: {time.time()-start_time}")

Total time: 281.76919770240784
